In [1]:
from libs.alibaba.visitor import Visitor

from libs.json import JSON
from openpyxl import load_workbook

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import os
import traceback
import types
import pendulum

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
visitor = Visitor(market, headless=False)

. storage markets.json
.\storage\markets.json was deserialized!


In [5]:
visitor.update()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy
E:\glitter\Eyelashes_config\visitors\visitors_2018-09-10.json was serialized!


In [ ]:
visitor.update = types.MethodType(update, visitor)

In [ ]:
visitors = visitor.deserialize()

In [8]:
len(visitors)

663

In [14]:
kws = {}
words = {}
expected_kws = ['volume', 'cluster', 'extension', 'flare', 'lash']
# expected_kws = ['magnet']
for v in visitors:
    if v['region'].lower() == 'china':
        continue
        
    for kw in v['keywords']:
        kw = kw.lower()
        kw = re.sub('eyelashes', 'eyelash', kw)
        kw = re.sub('lashes', 'lash', kw)
        kw = re.sub('extensions', 'extension', kw)
        
        for w in kw.split(' '):
            if w not in words:
                words[w] = 0
            words[w] = words[w] + 1
            
        for ekw in expected_kws:
            if ekw in kw:
                if kw not in kws:
                    kws[kw] = 0
                kws[kw] = kws[kw] + 1
                break

word_list = []
for w in words:
    word_list.append([w, words[w]])
sorted_words = sorted(word_list, key=lambda k: k[1], reverse=True)

kw_list = []
for kw in kws:
    kw_list.append([kw, kws[kw]])
    
sorted_kws = sorted(kw_list, key=lambda k: k[1], reverse=True)
for kw in sorted_kws:
    print(kw[1], kw[0])

78 mink eyelash
50 eyelash
32 eyelash extension
31 private label eyelash
28 3d mink eyelash
27 magnetic eyelash
16 mink eyelash private label
14 lash
14 false eyelash
11 silk eyelash
7 mink lash
7 eyelash packaging box
5 wholesale mink eyelash
5 magnetic lash
4 eyelash samples
4 magnet eyelash
4 eyelash package box
4 eyelash packaging
3 custom eyelash packaging
3 own brand eyelash
3 eyelash mink
3 synthetic false lash
3 3d eyelash
3 3d mink lash
2 custom eyelash box
2 lash branding
2 private label mink eyelash
2 wholesale 3d mink eyelash
2 mink eyelash vendor
2 silk eyelash extension
2 eyelash magnetic
2 premium mink eyelash wholesale
2 hair extension packaging
2 hair extension
2 custom lash packaging mink lash
2 cluster eyelash
2 wholesale lash
2 korean eyelash
2 korean eyelash extension
2 clear band false eyelash synthetic
2 clear band synthetic lash
2 eyelash 3d mink
1 custom eyelash package
1 thin eyelash
1 mink eyelash ve...
1 free mink eyelash
1 magnetic eye lash
1 magnetized las

In [25]:
%store sorted_kws

Stored 'sorted_kws' (list)


In [15]:
import re

In [22]:
href = "//hz-productposting.alibaba.com/product/editing.htm?id=60762374394"
ali_id = re.search('\?id=(\d+)', href).group(1)

In [27]:
from openpyxl import load_workbook

In [28]:
file = 'products ranking overview.xlsx'
wb = load_workbook(file)
sheet = wb.worksheets[0]
idx = 0
for kw in sorted_kws:
    idx += 1
    cell = sheet.cell(row=idx,column=1)
    cell.value = kw[1]
    cell = sheet.cell(row=idx,column=2)
    cell.value = kw[0]
wb.save(file)